# Ames Housing Data Analysis (Test) - 2nd Attempt

# Problem Statement

Dataset contains information from the Ames Assessor’s Office used in computing assessed values for individual residential properties sold in Ames, IA from 2006 to 2010.

Using this dataset, we build a regression model to predict the selling prices of houses. For each Id in the test set, i will predict the value of the Sale Price variable.

In [200]:
# Import the necessary libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Set row display option
pd.set_option('display.max_rows' , 100)
# pd.set_option('display.max_columns' , 100)

# modeling imports
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV

# Adding the gridline in graph
sns.set_style('whitegrid')

# Load the Data

In [201]:
test = pd.read_csv('../dataset/test.csv')

In [202]:
# Look at the data
test.head(10)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD
5,333,923228370,160,RM,21.0,1890,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,6,2010,WD
6,1327,902427150,20,RM,52.0,8516,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,5,2008,WD
7,858,907202130,20,RL,NaN,9286,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,10,2009,WD
8,95,533208090,160,FV,39.0,3515,Pave,Pave,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,1,2010,WD
9,1568,914476010,20,RL,75.0,10125,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,MnPrv,NaN,0,2,2008,WD


In [203]:
# Look at the data
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               879 non-null    int64  
 1   PID              879 non-null    int64  
 2   MS SubClass      879 non-null    int64  
 3   MS Zoning        879 non-null    object 
 4   Lot Frontage     719 non-null    float64
 5   Lot Area         879 non-null    int64  
 6   Street           879 non-null    object 
 7   Alley            58 non-null     object 
 8   Lot Shape        879 non-null    object 
 9   Land Contour     879 non-null    object 
 10  Utilities        879 non-null    object 
 11  Lot Config       879 non-null    object 
 12  Land Slope       879 non-null    object 
 13  Neighborhood     879 non-null    object 
 14  Condition 1      879 non-null    object 
 15  Condition 2      879 non-null    object 
 16  Bldg Type        879 non-null    object 
 17  House Style     

In [204]:
# Look at the data
test.describe()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold
count,879.000000,8.790000e+02,879.000000,719.000000,879.000000,879.000000,879.000000,879.000000,879.000000,878.000000,...,879.000000,879.000000,879.000000,879.000000,879.000000,879.000000,879.000000,879.000000,879.000000,879.000000
mean,1445.588168,7.165050e+08,58.270762,69.630042,10340.920364,6.054608,5.565415,1970.533561,1984.444824,106.982916,...,470.832765,93.560865,47.478953,24.037543,2.594994,14.813424,1.882821,48.443686,6.207053,2007.824801
std,850.717105,1.889135e+08,42.211389,23.625372,10047.335167,1.374756,1.128422,30.403527,20.454546,188.356829,...,213.070155,121.174306,69.209179,73.212237,24.948416,52.975963,29.899698,549.858353,2.644097,1.327396
min,2.000000,5.263021e+08,20.000000,21.000000,1477.000000,2.000000,1.000000,1880.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,692.500000,5.284861e+08,20.000000,59.000000,7298.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,...,323.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,1435.000000,5.354542e+08,50.000000,68.000000,9453.000000,6.000000,5.000000,1972.000000,1992.000000,0.000000,...,473.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2197.000000,9.071921e+08,70.000000,80.000000,11606.500000,7.000000,6.000000,2000.000000,2003.000000,173.500000,...,576.000000,171.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2928.000000,1.007100e+09,190.000000,182.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1378.000000,...,1488.000000,690.000000,742.000000,1012.000000,360.000000,576.000000,555.000000,15500.000000,12.000000,2010.000000


# Data Cleaning

Check for Null Values.

In [205]:
# Check for Null Values
test.isnull().sum()

Id                   0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       160
Lot Area             0
Street               0
Alley              821
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type         1
Mas Vnr Area         1
Exter Qual           0
Exter Cond           0
Foundation           0
Bsmt Qual           25
Bsmt Cond           25
Bsmt Exposure       25
BsmtFin Type 1      25
BsmtFin SF 1         0
BsmtFin Type 2      25
BsmtFin SF 2         0
Bsmt Unf SF          0
Total Bsmt SF        0
Heating              0
Heating QC           0
Central Air          0
Electrical 

We will apply the similar techniques as our train datasets, we will drop the following variables: `Alley`, `Fireplace Qu`, `Pool QC`, `Fence` & `Misc Feature`

In [206]:
# Initial data size
test.shape

(879, 80)

In [207]:
# Dropping the 5 columns
test.drop(columns=['Alley' , 'Fireplace Qu' , 'Pool QC' , 'Fence' , 'Misc Feature'],axis=1,inplace=True)

In [208]:
# New data size
test.shape

(879, 75)

In [209]:
# Check for nulls after dropping 5 columns
test.isnull().sum()

Id                   0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       160
Lot Area             0
Street               0
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type         1
Mas Vnr Area         1
Exter Qual           0
Exter Cond           0
Foundation           0
Bsmt Qual           25
Bsmt Cond           25
Bsmt Exposure       25
BsmtFin Type 1      25
BsmtFin SF 1         0
BsmtFin Type 2      25
BsmtFin SF 2         0
Bsmt Unf SF          0
Total Bsmt SF        0
Heating              0
Heating QC           0
Central Air          0
Electrical           1
1st Flr SF 

Filling up other missing values, replace those missing values after checking it.

Handling of `Lot Frontage` null data.

In [210]:
# Check the number of elements inside 'Lot Frontage'
test['Lot Frontage'].value_counts(dropna=False)

NaN      160
60.0      97
80.0      43
75.0      37
70.0      37
        ... 
174.0      1
122.0      1
150.0      1
31.0       1
133.0      1
Name: Lot Frontage, Length: 105, dtype: int64

In [211]:
# Replace the missing values with the maximum occurance.
test['Lot Frontage'] = test['Lot Frontage'].fillna(test['Lot Frontage'].mode()[0])
test['Lot Frontage'].value_counts(dropna=False)

60.0     257
80.0      43
70.0      37
75.0      37
50.0      27
        ... 
122.0      1
150.0      1
31.0       1
112.0      1
133.0      1
Name: Lot Frontage, Length: 104, dtype: int64

Handling of `Mas Vnr Type` null data.

In [212]:
# Check the number of elements inside 'Mas Vnr Type'
test['Mas Vnr Type'].value_counts(dropna=False)

None       534
BrkFace    250
Stone       81
BrkCmn      12
CBlock       1
NaN          1
Name: Mas Vnr Type, dtype: int64

In [213]:
# Replace the missing values with None (Most likely the house has no masonry veneer)
test['Mas Vnr Type'] = test['Mas Vnr Type'].fillna('None')
test['Mas Vnr Type'].value_counts(dropna=False)

None       535
BrkFace    250
Stone       81
BrkCmn      12
CBlock       1
Name: Mas Vnr Type, dtype: int64

Handling of `Mas Vnr Area` null data.

In [214]:
# Check the number of elements inside 'Mas Vnr Area'
test['Mas Vnr Area'].value_counts(dropna=False)

0.0      532
216.0      7
80.0       5
196.0      5
420.0      5
        ... 
464.0      1
NaN        1
312.0      1
101.0      1
264.0      1
Name: Mas Vnr Area, Length: 234, dtype: int64

In [215]:
# Replace the missing values with 0 (Most likely the house has no masonry veneer)
test['Mas Vnr Area'] = test['Mas Vnr Area'].fillna(test['Mas Vnr Area'].mode()[0])
test['Mas Vnr Area'].value_counts(dropna=False)

0.0      533
216.0      7
80.0       5
196.0      5
420.0      5
        ... 
464.0      1
312.0      1
101.0      1
238.0      1
264.0      1
Name: Mas Vnr Area, Length: 233, dtype: int64

Handling of `Bsmt Qual` null data

In [216]:
# Check the number of elements inside 'Bsmt Qual'
test['Bsmt Qual'].value_counts(dropna=False)

TA     396
Gd     355
Ex      74
Fa      28
NaN     25
Po       1
Name: Bsmt Qual, dtype: int64

In [217]:
# Replace the missing values with TA (as this is the maximum)
test['Bsmt Qual'] = test['Bsmt Qual'].fillna('TA')
test['Bsmt Qual'].value_counts(dropna=False)

TA    421
Gd    355
Ex     74
Fa     28
Po      1
Name: Bsmt Qual, dtype: int64

Handling of `Bsmt Cond` null data.

In [218]:
# Check the number of elements inside 'Bsmt Cond'
test['Bsmt Cond'].value_counts(dropna=False)

TA     782
Fa      39
Gd      33
NaN     25
Name: Bsmt Cond, dtype: int64

In [219]:
# Replace the missing values with TA (as this is the maximum)
test['Bsmt Cond'] = test['Bsmt Cond'].fillna('TA')
test['Bsmt Cond'].value_counts(dropna=False)

TA    807
Fa     39
Gd     33
Name: Bsmt Cond, dtype: int64

Handling of `Bsmt Exposure` null data.

In [220]:
# Check the number of elements inside 'Bsmt Exposure'
test['Bsmt Exposure'].value_counts(dropna=False)

No     567
Av     130
Gd      81
Mn      76
NaN     25
Name: Bsmt Exposure, dtype: int64

In [221]:
# Replace the missing values with No (as this is the maximum)
test['Bsmt Exposure'] = test['Bsmt Exposure'].fillna(test['Bsmt Exposure'].mode()[0])
test['Bsmt Exposure'].value_counts(dropna=False)

No    592
Av    130
Gd     81
Mn     76
Name: Bsmt Exposure, dtype: int64

Handling of `BsmtFin Type 1` null data.

In [222]:
# Check the number of elements inside 'Bsmt Type 1'
test['BsmtFin Type 1'].value_counts(dropna=False)

Unf    248
GLQ    244
ALQ    136
Rec    105
BLQ     69
LwQ     52
NaN     25
Name: BsmtFin Type 1, dtype: int64

In [223]:
# Replace the missing values with NA (this part is empty most likely because of no basement)
test['BsmtFin Type 1'] = test['BsmtFin Type 1'].fillna('NA')
test['BsmtFin Type 1'].value_counts(dropna=False)

Unf    248
GLQ    244
ALQ    136
Rec    105
BLQ     69
LwQ     52
NA      25
Name: BsmtFin Type 1, dtype: int64

Handling of `BsmtFin Type 2` null data.

In [224]:
# Check the number of elements inside 'BsmtFin Type 2'
test['BsmtFin Type 2'].value_counts(dropna=False)

Unf    750
LwQ     29
Rec     26
NaN     25
BLQ     20
ALQ     18
GLQ     11
Name: BsmtFin Type 2, dtype: int64

In [225]:
# Replace the missing values with NA (this part is empty most likely because of no basement)
test['BsmtFin Type 2'] = test['BsmtFin Type 2'].fillna('NA')
test['BsmtFin Type 2'].value_counts(dropna=False)

Unf    750
LwQ     29
Rec     26
NA      25
BLQ     20
ALQ     18
GLQ     11
Name: BsmtFin Type 2, dtype: int64

Handling of `Electrical` null data.

In [226]:
# Check the number of elements inside 'Electrical'
test['Electrical'].value_counts(dropna=False)

SBrkr    814
FuseA     48
FuseF     15
FuseP      1
NaN        1
Name: Electrical, dtype: int64

In [227]:
# Replace the missing values with SBrkr (as this is the maximum)
test['Electrical'] = test['Electrical'].fillna(test['Electrical'].mode()[0])
test['Electrical'].value_counts(dropna=False)

SBrkr    815
FuseA     48
FuseF     15
FuseP      1
Name: Electrical, dtype: int64

Handling of `Garage Type` null data.

In [228]:
# Check the number of elements inside 'Garage Type'
test['Garage Type'].value_counts(dropna=False)

Attchd     518
Detchd     246
BuiltIn     54
NaN         44
Basment      9
2Types       4
CarPort      4
Name: Garage Type, dtype: int64

In [229]:
# Replace the missing values with Attchd (as this is the maximum)
test['Garage Type'] = test['Garage Type'].fillna('Attchd')
test['Garage Type'].value_counts(dropna=False)

Attchd     562
Detchd     246
BuiltIn     54
Basment      9
2Types       4
CarPort      4
Name: Garage Type, dtype: int64

Handling of `Garage Yr Blt` null data.

In [230]:
# Check the number of elements inside 'Garage Yr Blt'
test['Garage Yr Blt'].value_counts(dropna=False)

NaN       45
2005.0    37
2006.0    35
2007.0    31
2004.0    27
2003.0    26
1977.0    20
1950.0    19
1997.0    18
2008.0    17
1968.0    16
1974.0    16
1993.0    16
1960.0    16
1999.0    15
1976.0    15
1998.0    14
2000.0    14
1980.0    14
1994.0    14
1969.0    14
1957.0    14
2001.0    14
1959.0    13
2002.0    13
1920.0    12
1970.0    12
1963.0    12
2009.0    12
1954.0    12
1961.0    11
1978.0    11
1964.0    10
1962.0    10
1972.0    10
1958.0    10
1966.0    10
1967.0    10
1979.0    10
1956.0    10
1995.0     9
1973.0     9
1996.0     9
1955.0     9
1925.0     9
1984.0     8
1989.0     7
1991.0     7
1930.0     7
1965.0     7
1926.0     7
1952.0     6
1985.0     6
1971.0     6
1988.0     6
1910.0     5
1981.0     5
1990.0     5
1938.0     5
1940.0     5
1939.0     5
1992.0     5
1951.0     5
1948.0     5
1949.0     4
1953.0     4
1983.0     4
1946.0     4
1941.0     4
1924.0     4
1900.0     3
1986.0     3
1915.0     3
2010.0     3
1982.0     3
1975.0     3
1922.0     2

In [231]:
# Replace the missing values with 2005 (as this is the maximum)
test['Garage Yr Blt'] = test['Garage Yr Blt'].fillna(test['Garage Yr Blt'].mode()[0])
test['Garage Yr Blt'].value_counts(dropna=False)

2005.0    82
2006.0    35
2007.0    31
2004.0    27
2003.0    26
1977.0    20
1950.0    19
1997.0    18
2008.0    17
1993.0    16
1974.0    16
1968.0    16
1960.0    16
1976.0    15
1999.0    15
1957.0    14
2001.0    14
1994.0    14
1969.0    14
1998.0    14
1980.0    14
2000.0    14
1959.0    13
2002.0    13
1963.0    12
2009.0    12
1970.0    12
1920.0    12
1954.0    12
1961.0    11
1978.0    11
1967.0    10
1962.0    10
1956.0    10
1979.0    10
1958.0    10
1964.0    10
1972.0    10
1966.0    10
1996.0     9
1995.0     9
1973.0     9
1955.0     9
1925.0     9
1984.0     8
1926.0     7
1965.0     7
1989.0     7
1991.0     7
1930.0     7
1952.0     6
1985.0     6
1971.0     6
1988.0     6
1910.0     5
1981.0     5
1990.0     5
1992.0     5
1938.0     5
1940.0     5
1939.0     5
1951.0     5
1948.0     5
1949.0     4
1953.0     4
1983.0     4
1946.0     4
1941.0     4
1924.0     4
2010.0     3
1975.0     3
1982.0     3
1915.0     3
1986.0     3
1900.0     3
1922.0     2
1947.0     2

Handling of `Garage Finish` null data.

In [232]:
# Check the number of elements inside 'Garage Finish'
test['Garage Finish'].value_counts(dropna=False)

Unf    382
RFn    233
Fin    219
NaN     45
Name: Garage Finish, dtype: int64

In [233]:
# Replace the missing values with Unf (as this is the maximum)
test['Garage Finish'] = test['Garage Finish'].fillna(test['Garage Finish'].mode()[0])
test['Garage Finish'].value_counts(dropna=False)

Unf    427
RFn    233
Fin    219
Name: Garage Finish, dtype: int64

Handling of `Garage Qual` null data.

In [234]:
# Check the number of elements inside 'Garage Qual'
test['Garage Qual'].value_counts(dropna=False)

TA     783
NaN     45
Fa      42
Gd       6
Po       3
Name: Garage Qual, dtype: int64

In [235]:
# Replace the missing values with TA (as this is the maximum)
test['Garage Qual'] = test['Garage Qual'].fillna(test['Garage Qual'].mode()[0])
test['Garage Qual'].value_counts(dropna=False)

TA    828
Fa     42
Gd      6
Po      3
Name: Garage Qual, dtype: int64

Handling of `Garage Cond` null data.

In [236]:
# Check the number of elements inside 'Garage Cond'
test['Garage Cond'].value_counts(dropna=False)

TA     797
NaN     45
Fa      27
Po       6
Gd       3
Ex       1
Name: Garage Cond, dtype: int64

In [237]:
# Replace the missing values with TA (as this is the maximum)
test['Garage Cond'] = test['Garage Cond'].fillna(test['Garage Cond'].mode()[0])
test['Garage Cond'].value_counts(dropna=False)

TA    842
Fa     27
Po      6
Gd      3
Ex      1
Name: Garage Cond, dtype: int64

In [238]:
# Check if there is anymore null values
test.isnull().sum()

Id                 0
PID                0
MS SubClass        0
MS Zoning          0
Lot Frontage       0
Lot Area           0
Street             0
Lot Shape          0
Land Contour       0
Utilities          0
Lot Config         0
Land Slope         0
Neighborhood       0
Condition 1        0
Condition 2        0
Bldg Type          0
House Style        0
Overall Qual       0
Overall Cond       0
Year Built         0
Year Remod/Add     0
Roof Style         0
Roof Matl          0
Exterior 1st       0
Exterior 2nd       0
Mas Vnr Type       0
Mas Vnr Area       0
Exter Qual         0
Exter Cond         0
Foundation         0
Bsmt Qual          0
Bsmt Cond          0
Bsmt Exposure      0
BsmtFin Type 1     0
BsmtFin SF 1       0
BsmtFin Type 2     0
BsmtFin SF 2       0
Bsmt Unf SF        0
Total Bsmt SF      0
Heating            0
Heating QC         0
Central Air        0
Electrical         0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
Gr Liv Area        0
Bsmt Full Bat

After confirmation, all the Null values of the data have been cleaned and replaced.

# Check my final model

SalePrice = 183260 + 
            (15001 * Overall Qual) + (21031 * Gr Liv Area) + (9196 * Total Bsmt SF) + (5753 * Garage Area) + 
            (2684 * Year Built) + (3509 * Year Remod/Add) + (7442 * Mas Vnr Area) + (3302 * Fireplaces) + 
            (10195 * BsmtFin SF 1) + (4166 * Lot Frontage) + (5694 * Lot Area) - (6506 * Exter Qual) - 
            (5762 * Kitchen Qual) - (3815 * Bsmt Qual) + (3273 * Exterior 1st_BrkFace) + (2508 * Exterior 1st_CemntBd) + 
            (2362 * Neighborhood_Crawfor) + (2516 * Neighborhood_NoRidge) + (6739 * Neighborhood_NridgHt) + (7872 * Neighborhood_StoneBr) - 
            (2213 * Mas Vnr Type_BrkCmn) - (4117 * Mas Vnr Type_BrkFace)

In [239]:
# Create a new column as SalePrice
test['SalePrice'] = 0

In [240]:
# Check the new column
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,2658,902301120,190,RM,69.0,9142,Pave,Reg,Lvl,AllPub,...,60,112,0,0,0,0,4,2006,WD,0
1,2718,905108090,90,RL,60.0,9662,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,0,8,2006,WD,0
2,2414,528218130,60,RL,58.0,17104,Pave,IR1,Lvl,AllPub,...,24,0,0,0,0,0,9,2006,New,0
3,1989,902207150,30,RM,60.0,8520,Pave,Reg,Lvl,AllPub,...,0,184,0,0,0,0,7,2007,WD,0
4,625,535105100,20,RL,60.0,9500,Pave,IR1,Lvl,AllPub,...,76,0,0,185,0,0,7,2009,WD,0


# Map the non-numerical data to numbers

Mapping the `Exter Qual` data.

In [241]:
test['Exter Qual'].value_counts(dropna=False)

TA    552
Gd    292
Ex     26
Fa      9
Name: Exter Qual, dtype: int64

In [242]:
Exter_dict = {'Ex':1 , 'Gd':2 , 'TA':3 , 'Fa':4}

In [243]:
test['Exter Qual'] = test['Exter Qual'].map(Exter_dict)

In [244]:
test['Exter Qual'].value_counts(dropna=False)

3    552
2    292
1     26
4      9
Name: Exter Qual, dtype: int64

Mapping the `Kitchen Qual` data.

In [245]:
test['Kitchen Qual'].value_counts(dropna=False)

TA    447
Gd    354
Ex     54
Fa     23
Po      1
Name: Kitchen Qual, dtype: int64

In [246]:
Kitchen_dict = {'Ex':1 , 'Gd':2 , 'TA':3 , 'Fa':4 , 'Po':5}

In [247]:
test['Kitchen Qual'] = test['Kitchen Qual'].map(Kitchen_dict)

In [248]:
test['Kitchen Qual'].value_counts(dropna=False)

3    447
2    354
1     54
4     23
5      1
Name: Kitchen Qual, dtype: int64

Mapping the `Bsmt Qual` data.

In [249]:
test['Bsmt Qual'].value_counts(dropna=False)

TA    421
Gd    355
Ex     74
Fa     28
Po      1
Name: Bsmt Qual, dtype: int64

In [250]:
Bsmt_dict = {'Ex':1 , 'Gd':2 , 'TA':3 , 'Fa':4 , 'Po':5}

In [251]:
test['Bsmt Qual'] = test['Bsmt Qual'].map(Bsmt_dict)

In [252]:
test['Bsmt Qual'].value_counts(dropna=False)

3    421
2    355
1     74
4     28
5      1
Name: Bsmt Qual, dtype: int64

# Build a same model as our training data

In [253]:
X = test[['Overall Qual' , 'Gr Liv Area' , 'Total Bsmt SF' , 'Garage Area' , 
          'Year Built' , 'Year Remod/Add' , 'Mas Vnr Area' , 'Fireplaces' ,
          'BsmtFin SF 1' , 'Lot Frontage' , 'Lot Area' , 'Exter Qual' ,
          'Kitchen Qual' , 'Bsmt Qual' , 'Exterior 1st' , 'Neighborhood' ,
          'Mas Vnr Type' ]]

In [254]:
# Get_Dummies for nominal variables
X = pd.get_dummies(X, columns=['Exterior 1st' , 'Neighborhood' , 'Mas Vnr Type'])

In [255]:
y = test[['Id' , 'SalePrice']]

In [256]:
# Check the head of X
X.head()

,Overall Qual,Gr Liv Area,Total Bsmt SF,Garage Area,Year Built,Year Remod/Add,Mas Vnr Area,Fireplaces,BsmtFin SF 1,Lot Frontage,...,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_None,Mas Vnr Type_Stone
0,6,1928,1020,440,1910,1950,0.0,0,0,69.0,...,0,0,0,0,0,0,0,0,1,0
1,5,1967,1967,580,1977,1977,0.0,0,0,60.0,...,0,0,0,0,0,0,0,0,1,0
2,7,1496,654,426,2006,2006,0.0,1,554,58.0,...,0,0,0,0,0,0,0,0,1,0
3,5,968,968,480,1923,2006,0.0,0,0,60.0,...,0,0,0,0,0,0,0,0,1,0
4,6,1394,1394,514,1963,1963,247.0,2,609,60.0,...,0,0,0,0,0,0,1,0,0,0


In [257]:
# Check the head of X
X.columns

Index(['Overall Qual', 'Gr Liv Area', 'Total Bsmt SF', 'Garage Area',
       'Year Built', 'Year Remod/Add', 'Mas Vnr Area', 'Fireplaces',
       'BsmtFin SF 1', 'Lot Frontage', 'Lot Area', 'Exter Qual',
       'Kitchen Qual', 'Bsmt Qual', 'Exterior 1st_AsbShng',
       'Exterior 1st_AsphShn', 'Exterior 1st_BrkComm', 'Exterior 1st_BrkFace',
       'Exterior 1st_CemntBd', 'Exterior 1st_HdBoard', 'Exterior 1st_MetalSd',
       'Exterior 1st_Plywood', 'Exterior 1st_PreCast', 'Exterior 1st_Stucco',
       'Exterior 1st_VinylSd', 'Exterior 1st_Wd Sdng', 'Exterior 1st_WdShing',
       'Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale',
       'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert',
       'Neighborhood_Greens', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV',
       'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill',
       'Neighborhood_NWAmes', '

By checking the X columns, we realize that Neighborhood_GrnHill is not in the test data, so we may remove this from our model.

In [258]:
# Check the head of y
y.head()

,Id,SalePrice
0,2658,0
1,2718,0
2,2414,0
3,1989,0
4,625,0


# Calculate the SalePrice according to the formula

In [277]:
y['SalePrice'] =  183260 + 15001 * X['Overall Qual'] + 21036 * X['Gr Liv Area'] + 9196 * X['Total Bsmt SF'] + 5753 * X['Garage Area'] + 2684 * X['Year Built'] + 3509 * X['Year Remod/Add'] + 7442 * X['Mas Vnr Area'] + 3302 * X['Fireplaces'] + 10195 * X['BsmtFin SF 1'] + 4166 * X['Lot Frontage'] + 5694 * X['Lot Area'] - 6506 * X['Exter Qual'] - 5762 * X['Kitchen Qual'] - 3815 * X['Bsmt Qual'] + 3273 * X['Exterior 1st_BrkFace'] + 2508 * X['Exterior 1st_CemntBd'] + 2362 * X['Neighborhood_Crawfor'] + 2516 * X['Neighborhood_NoRidge'] + 6739 * X['Neighborhood_NridgHt'] + 7872 * X['Neighborhood_StoneBr'] - 2213 * X['Mas Vnr Type_BrkCmn'] - 4117 * X['Mas Vnr Type_BrkFace']


<ipython-input-277-a60fbfc3794b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['SalePrice'] =  183260 + 15001 * X['Overall Qual'] + 21036 * X['Gr Liv Area'] + 9196 * X['Total Bsmt SF'] + 5753 * X['Garage Area'] + 2684 * X['Year Built'] + 3509 * X['Year Remod/Add'] + 7442 * X['Mas Vnr Area'] + 3302 * X['Fireplaces'] + 10195 * X['BsmtFin SF 1'] + 4166 * X['Lot Frontage'] + 5694 * X['Lot Area'] - 6506 * X['Exter Qual'] - 5762 * X['Kitchen Qual'] - 3815 * X['Bsmt Qual'] + 3273 * X['Exterior 1st_BrkFace'] + 2508 * X['Exterior 1st_CemntBd'] + 2362 * X['Neighborhood_Crawfor'] + 2516 * X['Neighborhood_NoRidge'] + 6739 * X['Neighborhood_NridgHt'] + 7872 * X['Neighborhood_StoneBr'] - 2213 * X['Mas Vnr Type_BrkCmn'] - 4117 * X['Mas Vnr Type_BrkFace']


In [278]:
y.head()

,Id,SalePrice
0,2658,116995080.0
1,2718,130525864.0
2,2414,155897213.0
3,1989,93205764.0
4,625,119878517.0


In [279]:
# Set row display option
pd.set_option('display.max_rows' , None)
pd.set_option('display.max_columns' , None)

In [280]:
y.head()

,Id,SalePrice
0,2658,116995080.0
1,2718,130525864.0
2,2414,155897213.0
3,1989,93205764.0
4,625,119878517.0


In [ ]:
y.to_csv('../dataset/trial_two.csv',index=False)